# Surface adjusted distance project

Start by loading up some packages. 

## Note

Before running this code, use the Clusters tab to provision some resources (multiple cpus!).

In [1]:
import sys
print (sys.version)
import time
import itertools
import cProfile, pstats
import os.path
import numpy as np
import pandas as pd
from ipyparallel import Client
import surfaceAdjusted

# next line loads packages installed for my user account
sys.path.append("C:/ProgramData/Anaconda3/lib/site-packages")

3.7.3 (default, Mar 27 2019, 17:13:21) [MSC v.1915 64 bit (AMD64)]


C:\ProgramData\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\ProgramData\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Verify that the cluster is up and running

In [2]:
# check cluster status
rc = Client()
print(rc.ids)

[0, 1, 2, 3]


In [3]:
lview = rc.load_balanced_view()
lview.block = True
print(lview)

<LoadBalancedView None>


In [4]:
#test it 
lview.map(lambda x:x**10, range(8))

[0, 1, 1024, 59049, 1048576, 9765625, 60466176, 282475249]

Now use `%px` to import packages on each of the processors.

In [5]:
%px import sys
%px sys.path.append("C:/ProgramData/Anaconda3/lib/site-packages")
%px sys.path.append("C:/Users/yi/git/TerrainMetrics_conda2/Update")

# load surfaceAdjusted module on each worker
%px import surfaceAdjusted

[stderr:0] 
C:\ProgramData\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\ProgramData\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)
[stderr:1] 
C:\ProgramData\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\ProgramData\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.py

## Determining the cases 

We need to generate a data frame containing a row for each distance calculation we want to do. 
The following code blocks assign the cases of interest, and generate all relevant combinations, producing a data frame at the end.

In [3]:
# for testing, just use the larger resolutions
resolution_L = [100, 1000] # [3,10,30,100,1000]

# NN interpolation does not work, leave commented out
methods = ['clos', 'wavg', 'biLin', 'biQua', 'biQub', 'TIN', 'p2p', 'NN']

# for testing, just use one state
study_areas = ["Colorado"] #["Colorado", "Nebraska", "Louisiana", "Washington", "NC", "Texas"]

data_dir = r"D:/SAD/Modified_2/"

In [4]:
def expandgrid(*itrs):
    """
    Generate all possible combinations of elements in multiple lists
    
    Args:
        - lists separated by commas
    
    Example:
        >>> expandgrid([0, 1], [2, 3, 4])
        >>> [[0, 0, 0, 1, 1, 1], [2, 3, 4, 2, 3, 4]]
    
    Returns: 
        - a list of lists, with one element in each inner list for each 
          combination of elements in the input lists
    """
    product = list(itertools.product(*itrs))
    return [[x[i] for x in product] for i in range(len(itrs))]

In [5]:
# Determine all of the cases to compute (area X resolution X transect X method combinations)
cases_list = []
for area in study_areas:
    area_start_time = time.time()
    area_path = data_dir + area + '/simulation/'
    area_transects = np.genfromtxt(area_path + 'tran_sim_pts.csv', delimiter=",")
    
    for resolution in resolution_L:
        
        n_transects = int(area_transects.shape[0] / 2)
        transect_indices = [i for i in range(n_transects)]

        if resolution == 3:
            # subset 3m resolution to "clos" method only
            cases = expandgrid(transect_indices, ["clos"], [resolution], [area_path], [area])
        else:
            # determine all possible combinations of transects and methods
            cases = expandgrid(transect_indices, methods, [resolution], [area_path], [area])
        
        n_cases = len(cases[0])
        
        df = pd.DataFrame(cases).transpose()
        df.columns = ["transect", "method", "resolution", "path", "area"]
        cases_list.append(df)

cases_df = pd.concat(cases_list)
cases_df.describe()

,transect,method,resolution,path,area
count,16000,16000,16000,16000,16000
unique,1000,8,2,1,1
top,999,biQua,1000,D:/SAD/Modified_2/Colorado/simulation/,Colorado
freq,16,2000,8000,16000,16000


For testing purposes, we'll just use a few of the transects.

In [6]:
# subset transects
n_transects = 2
cases_df = cases_df.loc[lambda df: df.transect < n_transects, :]
cases_df

,transect,method,resolution,path,area
0,0,clos,100,D:/SAD/Modified_2/Colorado/simulation/,Colorado
1,0,wavg,100,D:/SAD/Modified_2/Colorado/simulation/,Colorado
2,0,biLin,100,D:/SAD/Modified_2/Colorado/simulation/,Colorado
3,0,biQua,100,D:/SAD/Modified_2/Colorado/simulation/,Colorado
4,0,biQub,100,D:/SAD/Modified_2/Colorado/simulation/,Colorado
5,0,TIN,100,D:/SAD/Modified_2/Colorado/simulation/,Colorado
6,0,p2p,100,D:/SAD/Modified_2/Colorado/simulation/,Colorado
7,0,NN,100,D:/SAD/Modified_2/Colorado/simulation/,Colorado
8,1,clos,100,D:/SAD/Modified_2/Colorado/simulation/,Colorado
9,1,wavg,100,D:/SAD/Modified_2/Colorado/simulation/,Colorado


In [ ]:
# try just one case
surfaceAdjusted.distance(
                cases_df['transect'].tolist()[0], 
                cases_df['method'].tolist()[7], 
                cases_df['resolution'].tolist()[3], 
                cases_df['path'].tolist()[0])

The next line maps our distance function to all of the cases with automatic load balancing. 

In [ ]:
# in parallel
print("Processing")
res = lview.map(surfaceAdjusted.distance, 
                cases_df['transect'].tolist(), 
                cases_df['method'].tolist(), 
                cases_df['resolution'].tolist(), 
                cases_df['path'].tolist())